## 5. Tool use(crewai)

<img style="float: right;" src="../img/flex-logo.png" width="120"><br>

<div style="text-align: right"> <b>your name</b></div>
<div style="text-align: right"> Initial issue : 2025.10.02 </div>
<div style="text-align: right"> last update : 2025.10.02 </div>

개정 이력  
- `2025.10.02` : 노트북 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
import os, getpass
import asyncio
import nest_asyncio
from typing import List, Optional
from dotenv import load_dotenv
import logging
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool as langchain_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [2]:
try:
    llm: Optional[ChatOpenAI] = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
except Exception as e:
    print(f"언어 모델 초기화 오류: {e}")
    llm = None

In [3]:
# try:
#   # A model with function/tool calling capabilities is required.
#   llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
#   print(f"✅ Language model initialized: {llm.model}")
# except Exception as e:
#   print(f"🛑 Error initializing language model: {e}")
#   llm = None

### 1. 도구 정의

In [4]:
@langchain_tool
def search_information(query: str) -> str:
  """
  주어진 주제에 대한 사실 정보를 제공합니다.
  '프랑스의 수도'나 '런던의 날씨?'와 같은 문장의 답을 찾을 때 이 도구를 사용하세요.
  """
  print(f"\n--- 🛠️ 도구 호출: search_information | 쿼리: '{query}' ---")
  # 사전 정의된 결과로 검색 도구를 모의(simulate)합니다.
  simulated_results = {
      "런던의 날씨": "현재 런던의 날씨는 흐리고 기온은 15°C입니다.",
      "프랑스의 수도": "프랑스의 수도는 파리입니다.",
      "지구의 인구": "지구의 추정 인구는 약 80억 명입니다.",
      "가장 높은 산": "해수면 기준 가장 높은 산은 에베레스트산입니다.",
      "default": f"'{query}'에 대한 모의 검색 결과: 구체적인 정보를 찾지 못했지만, 흥미로운 주제네요."
  }
  result = simulated_results.get(query.lower(), simulated_results["default"])
  print(f"--- 도구 결과: {result} ---")
  return result

tools = [search_information]


### 2. Tool calling agent 정의

In [5]:
if llm:
  # This prompt template requires an `agent_scratchpad` placeholder for the agent's internal steps.
  agent_prompt = ChatPromptTemplate.from_messages([
      ("system", "당신은 친절한 어시스턴트 입니다."),
      ("human", "{input}"),
      ("placeholder", "{agent_scratchpad}"),
  ])
  # Create the agent, binding the LLM, tools, and prompt together.
  agent = create_tool_calling_agent(llm, tools, agent_prompt)
  # AgentExecutor is the runtime that invokes the agent and executes the chosen tools.
  # The 'tools' argument is not needed here as they are already bound to the agent.
  agent_executor = AgentExecutor(agent=agent, verbose=True, tools=tools)

### 3. 실행 함수 정의/ 테스트

In [6]:
async def run_agent_with_tool(query: str):
  """쿼리로 에이전트 실행기를 호출하고 최종 응답을 출력합니다."""
  print(f"\n--- 🏃 에이전트 실행: 쿼리 '{query}' ---")
  try:
      response = await agent_executor.ainvoke({"input": query})
      print("\n--- ✅ 최종 에이전트 응답 ---")
      print(response["output"])
  except Exception as e:
      print(f"\n🛑 에이전트 실행 중 오류가 발생했습니다: {e}")



In [7]:
async def main():
  """Runs all agent queries concurrently."""
  tasks = [
      run_agent_with_tool("What is the capital of France?"),
      run_agent_with_tool("What's the weather like in London?"),
      run_agent_with_tool("Tell me something about dogs.")  # Should trigger the default tool response
  ]
  await asyncio.gather(*tasks)

In [8]:
nest_asyncio.apply()
asyncio.run(main())


--- 🏃 에이전트 실행: 쿼리 'What is the capital of France?' ---

--- 🏃 에이전트 실행: 쿼리 'What's the weather like in London?' ---

--- 🏃 에이전트 실행: 쿼리 'Tell me something about dogs.' ---


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `search_information` with `{'query': 'capital of France'}`



--- 🛠️ 도구 호출: search_information | 쿼리: 'capital of France' ---
--- 도구 결과: 'capital of France'에 대한 모의 검색 결과: 구체적인 정보를 찾지 못했지만, 흥미로운 주제네요. ---
'capital of France'에 대한 모의 검색 결과: 구체적인 정보를 찾지 못했지만, 흥미로운 주제네요.The capital of France is Paris.

> Finished chain.

--- ✅ 최종 에이전트 응답 ---
The capital of France is Paris.

Invoking: `search_information` with `{'query': 'facts about dogs'}`



--- 🛠️ 도구 호출: search_information | 쿼리: 'facts about dogs' ---
--- 도구 결과: 'facts about dogs'에 대한 모의 검색 결과: 구체적인 정보를 찾지 못했지만, 흥미로운 주제네요. ---
'facts about dogs'에 대한 모의 검색 결과: 구체적인 정보를 찾지 못했지만, 흥미로운 주제네요.
Invoking: `search_information` with `{'query': 'London 